<a href="https://colab.research.google.com/github/sisomimoctrl/study-pytorch/blob/main/multivariate_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from enum import Enum

torch.manual_seed(777)

"""
 0: 기본
 1: nn.Linear 사용
 2: model을 클래스로 구현
"""
method = 2

print(f'method : {method}\n')

# 데이터
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

lr = 1e-5 # learning rate를 의미하는 lr은 1e-5

if method == 0:
    # 모델을 선언 및 초기화. y_train = x_train*W + b
    W = torch.zeros((3, 1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    
    # optimizer 설정. 경사 하강법 SGD를 사용 
    optimizer = optim.SGD([W, b], lr=lr)

elif method == 1:
    # input_dim=1, output_dim=1.
    model = nn.Linear(3,1)

    # optimizer 설정. 경사 하강법 SGD를 사용
    optimizer = torch.optim.SGD(model.parameters(), lr=lr) 
    
elif method == 2:
    class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
        def __init__(self):
            super().__init__()
            self.linear = nn.Linear(3, 1) # input_dim=3, output_dim=1

        def forward(self, x):
            return self.linear(x)

    model = LinearRegressionModel()

    # optimizer 설정. 경사 하강법 SGD를 사용
    optimizer = torch.optim.SGD(model.parameters(), lr=lr) 
        
        
# 전체 훈련 데이터에 대해 경사 하강법을 3,000회 반복
nb_epochs = 3000
for epoch in range(nb_epochs + 1):

    if method == 0:
        # H(x) 계산
        hypothesis = x_train.matmul(W) + b
        
        # cost 계산
        cost = torch.mean((hypothesis - y_train)**2)
        
    else: # method=1, 2
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()
    
    if epoch % 300 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')
        
# 검증과정

# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 

# 파라미터 출력
if method == 0:
    pred_y = new_var.matmul(W) + b
    print(f'\nW: {W}\nb: {b}\n')
else:
    pred_y = model(new_var) # forward 연산
    print(f'\n{list(model.parameters())}\n')

print(f'훈련 후 입력이 {new_var}일 때의 예측값 : {pred_y.item():.6f}')

method : 2

Epoch    0/3000 Cost: 50106.699219
Epoch  300/3000 Cost: 9.395685
Epoch  600/3000 Cost: 8.401238
Epoch  900/3000 Cost: 7.515099
Epoch 1200/3000 Cost: 6.725065
Epoch 1500/3000 Cost: 6.020391
Epoch 1800/3000 Cost: 5.391678
Epoch 2100/3000 Cost: 4.830501
Epoch 2400/3000 Cost: 4.329469
Epoch 2700/3000 Cost: 3.881966
Epoch 3000/3000 Cost: 3.482246

[Parameter containing:
tensor([[0.7477, 0.7998, 0.4787]], requires_grad=True), Parameter containing:
tensor([-0.1181], requires_grad=True)]

훈련 후 입력이 tensor([[73., 80., 75.]])일 때의 예측값 : 154.348038
